In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dropping.csv')

In [3]:
df

,Name,DOB,Email
0,Shubham Kalantri,09-10-2002,f20201008@goa.bits-pilani.ac.in
1,Krishanu Shah,NaN,f20201728@goa.bits-pilani.ac.in
2,Saraswati Padmawar,08/10/02,NaN


In [4]:
df

,Name,DOB,Email
0,Shubham Kalantri,09-10-2002,f20201008@goa.bits-pilani.ac.in
1,Krishanu Shah,NaN,f20201728@goa.bits-pilani.ac.in
2,Saraswati Padmawar,08/10/02,NaN


In [32]:
class dataset:
    def __init__(self, file):
        self.file = file
        self.df = pd.read_csv(file)
    def dropRow(self, columns = []) -> None:
        if len(columns) == 0:
            self.df = self.df.dropna()
        else:
            self.df = self.df.dropna(subset = columns)
# dropColumn is a bit general, deopEmptyColumn is more specific
    def dropColumns(self, columns = []) -> None:
        self.df = self.df.drop(columns, axis = 1)
    def dropEmptyColumns(self) -> None:
        pass
    def stdDate(self, column = 'Date', format = 'ddmmyyyy') -> None:
        fmt = '%d/%m/%Y'
        if format.lower() in ['dd/mm/yyyy', 'dd-mm-yyyy', 'dd.mm.yyyy', 'ddmmyyyy', 'dd mm yyyy']:
            fmt = '%d/%m/%Y'
        elif format.lower() in ['dd/mm/yy', 'dd-mm-yy', 'dd.mm.yy', 'ddmmyy', 'dd mm yy']:
            fmt = '%d/%m/%y'
        elif format.lower() in ['mm/dd/yyyy', 'mm-dd-yyyy', 'mm.dd.yyyy', 'mmddyyyy', 'mm dd yyyy']:
            fmt = '%m/%d/%Y'
        elif format.lower() in ['mm/dd/yy', 'mm-dd-yy', 'mm.dd.yy', 'mmddyy', 'mm dd yy']:
            fmt = '%m/%d/%y'
        elif format.lower() in ['yyyy/mm/dd', 'yyyy-mm-dd', 'yyyy.mm.dd', 'yyyymmdd', 'yyyy mm dd']:
            fmt = '%Y/%m/%d'
        elif format.lower() in ['yy/mm/dd', 'yy-mm-dd', 'yy.mm.dd', 'yymmdd', 'yy mm dd']:
            fmt = '%y/%m/%d'
        self.df[column] = pd.to_datetime(self.df[column])
        self.df[column] = self.df[column].dt.strftime(fmt)

    def savetofile(self, file, type = 'csv') -> None:
        if type == 'csv':
            self.df.to_csv(file, index = False)
        elif type == 'xlsx':
            self.df.to_excel(file, index = False)
        elif type == 'json':
            self.df.to_json(file, orient = 'table', index = False)
    # function to remove duplicates across columns, or a group of columns
    
    def dropDuplicates(self, columns = []) -> None:
        if len(columns) == 0:
            self.df = self.df.drop_duplicates()
        else:
            self.df = self.df.drop_duplicates(subset = columns, keep='first')
    
    '''Normalise Numerical data in a dataframe. '''
    '''Can have different methods of normalization'''
    ''' yeah true, but I think it's better to have a function that can do both, and then have a function that can do one or the other. have a default method and then have a parameter that can be changed to do the other method. '''
    ''' abey copilot wrote upar waala lmfao so good actually giving good ideas bey. more useful than certain group members 😭'''
    def normalise(self, columns = []):
        if len(columns) == 0:
            self.df = (self.df - self.df.min()) / (self.df.max() - self.df.min())
        else:
            self.df[columns] = (self.df[columns] - self.df[columns].min()) / (self.df[columns].max() - self.df[columns].min())
    
    
    # def cleanName(self, column = 'Name') -> None:
    #     self.data[column] = self.data[column].str.replace(' ', '')
    #     for x in range(len(self.data[column])):
    #         i = 0
    #         while i < len(self.data[column][x]):
    #             if self.data[column][x][i].isupper() and i != 0:
    #                 self.data[column][x] = self.data[column][x][:i] + ' ' + self.data[column][x][i:]
    #                 i += 1
    #             i += 1

In [33]:
df = dataset('dropping.csv')
df.stdDate('DOB', format = 'yyddmm')

In [34]:
df.savetofile('dropping.xlsx', 'xlsx')
df.savetofile('dropping.json', 'json')